In [1]:
import pandas as pd
import numpy as np
import re
import xlrd
import math
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import *
import time

# Загрузка файла с данными

In [2]:
time_df = pd.read_excel(
    "1-Данные из 1С по всем заказам от 2020-02-27.xlsx", index_col=0)

In [3]:
BaseDataFrame = time_df.copy()

In [4]:
BaseDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148621 entries, 0 to 148620
Data columns (total 40 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   AktVipRabot                  148621 non-null  object
 1   Data                         148621 non-null  object
 2   SummaAktaVipRabot            148621 non-null  object
 3   Quantity                     148621 non-null  object
 4   KodStrokiAkt                 148621 non-null  object
 5   ZakazKlienta                 148621 non-null  object
 6   NomenklaturaKod              148621 non-null  object
 7   Nomenklatura                 148621 non-null  object
 8   ZakazData                    148621 non-null  object
 9   PartnerKod                   148621 non-null  object
 10  OsnovnoyManager              148574 non-null  object
 11  BiznesRegion                 144337 non-null  object
 12  SummaZakaza                  148621 non-null  object
 13  GrafikOplaty  

# Обработка числовых типов

In [5]:
ColumsForChange = ['SummaAktaVipRabot', 
                  'Quantity',
                  'KodStrokiAkt',
                  'SummaZakaza', 
                  'KodStrokiZakaz',
                  'ZakazanoShtuk',
                  'Cena',
                  'ProcentRuchSkidki',
                  'ProcentAvtomSkidki',
                  'SrokPostavki',
                  'SummaRealiz',
                  'ProdanoShtuk',
                  'KodStrokiRealiz']
for elem in ColumsForChange:
    BaseDataFrame[elem] = BaseDataFrame[elem].astype('str')
    BaseDataFrame[elem] = BaseDataFrame[elem].str.replace('None', '0')
    BaseDataFrame[elem] = BaseDataFrame[elem].str.replace('nan', '0')
    BaseDataFrame[elem] = BaseDataFrame[elem].str.replace(u'\xa0', u'')
    BaseDataFrame[elem] = BaseDataFrame[elem].str.replace(',', '.')
    BaseDataFrame[elem] = BaseDataFrame[elem].astype('float')

In [6]:
# Посчитаем сумму заказанного товара по строкам (Хотя можно взять из 1С)
BaseDataFrame["SummaZakazano"] = None
for number, series in BaseDataFrame.iterrows():
    BaseDataFrame.at[number, "SummaZakazano"] = series['ZakazanoShtuk'] * series['Cena'] / \
        100 * (100 - series['ProcentAvtomSkidki'] +
               series['ProcentRuchSkidki'])

In [7]:
f = {0:np.nan}
BaseDataFrame = BaseDataFrame.replace({'SummaAktaVipRabot': f})
BaseDataFrame = BaseDataFrame.replace({'Quantity': f})
BaseDataFrame = BaseDataFrame.replace({'KodStrokiAkt': f})
BaseDataFrame = BaseDataFrame.replace({'SummaZakaza': f})
BaseDataFrame = BaseDataFrame.replace({'KodStrokiZakaz': f})
BaseDataFrame = BaseDataFrame.replace({'ZakazanoShtuk': f})
BaseDataFrame = BaseDataFrame.replace({'Cena': f})
BaseDataFrame = BaseDataFrame.replace({'ProcentRuchSkidki': f})
BaseDataFrame = BaseDataFrame.replace({'ProcentAvtomSkidki': f})
BaseDataFrame = BaseDataFrame.replace({'SrokPostavki': f})
BaseDataFrame = BaseDataFrame.replace({'SummaRealiz': f})
BaseDataFrame = BaseDataFrame.replace({'ProdanoShtuk': f})
BaseDataFrame = BaseDataFrame.replace({'KodStrokiRealiz': f})
BaseDataFrame = BaseDataFrame.replace({'SummaZakazano': f})

In [8]:
BaseDataFrame[ColumsForChange].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148621 entries, 0 to 148620
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   SummaAktaVipRabot   2390 non-null    float64
 1   Quantity            2390 non-null    float64
 2   KodStrokiAkt        2390 non-null    float64
 3   SummaZakaza         88575 non-null   float64
 4   KodStrokiZakaz      148621 non-null  float64
 5   ZakazanoShtuk       148621 non-null  float64
 6   Cena                148601 non-null  float64
 7   ProcentRuchSkidki   22368 non-null   float64
 8   ProcentAvtomSkidki  20920 non-null   float64
 9   SrokPostavki        54481 non-null   float64
 10  SummaRealiz         81869 non-null   float64
 11  ProdanoShtuk        81869 non-null   float64
 12  KodStrokiRealiz     81869 non-null   float64
dtypes: float64(13)
memory usage: 19.9 MB


# Обработка дат

In [9]:
ColumsForChange = ['Data',
                   'ZakazData',
                   'JelaemayaDataOtgruzki',
                   'DataOtgruzki',
                   'DataRealiz']

for elem in ColumsForChange:
    BaseDataFrame[elem] = BaseDataFrame[elem].astype('str')
    BaseDataFrame[elem] = BaseDataFrame[elem].str.replace('None', '')

    for number, series in BaseDataFrame.iterrows():
        if len(series[elem]) > 0:
            BaseDataFrame.at[number, elem] = datetime.strptime(
                series[elem], "%d.%m.%Y %H:%M:%S")

    BaseDataFrame[elem] = pd.to_datetime(BaseDataFrame[elem], errors='coerce')

# сортировка по дате
BaseDataFrame = BaseDataFrame.sort_values(
    by=['ZakazData', 'LinkOrder'])  # Сортировка по СсылкаЗаказ и дате
# сбросим имеющиеся индексы и воссоздадим по порядку заново
BaseDataFrame.reset_index(inplace=True)
# удалим старые индексы полученые после сброса на пред шаге
del BaseDataFrame['index']

In [10]:
BaseDataFrame[ColumsForChange].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148621 entries, 0 to 148620
Data columns (total 5 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Data                   2390 non-null    datetime64[ns]
 1   ZakazData              148621 non-null  datetime64[ns]
 2   JelaemayaDataOtgruzki  141083 non-null  datetime64[ns]
 3   DataOtgruzki           145941 non-null  datetime64[ns]
 4   DataRealiz             81869 non-null   datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 5.7 MB


# Обработка строковых типов

In [11]:
f = {'None':None, 'nan': None}
ColumsForChange = ['AktVipRabot', 
                   'ZakazKlienta',
                   'NomenklaturaKod',
                   'Nomenklatura',
                   'PartnerKod',
                   'OsnovnoyManager',
                   'BiznesRegion',
                   'GrafikOplaty',
                   'KreditIliPredoplata',
                   'StatusZakaza',
                   'SposobDostavki',
                   'NomenklaturaKodRealAndZakaz',
                   'NomenklaturaRealizAndZakaz',
                   'VidNomenklatury',
                   'KodTNVED',
                   'VidCeny',
                   'StavkaNDS',
                   'PrichinaOtmeny',
                   'Otmeneno',
                   'VariantObecpech',
                   'RealizSsilka',
                   'LinkOrder']
for elem in ColumsForChange:
    BaseDataFrame[elem] = BaseDataFrame[elem].astype('str')
    BaseDataFrame = BaseDataFrame.replace({elem: f})

In [12]:
BaseDataFrame[ColumsForChange].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148621 entries, 0 to 148620
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype 
---  ------                       --------------   ----- 
 0   AktVipRabot                  2390 non-null    object
 1   ZakazKlienta                 2390 non-null    object
 2   NomenklaturaKod              2390 non-null    object
 3   Nomenklatura                 2390 non-null    object
 4   PartnerKod                   148621 non-null  object
 5   OsnovnoyManager              148499 non-null  object
 6   BiznesRegion                 144337 non-null  object
 7   GrafikOplaty                 147813 non-null  object
 8   KreditIliPredoplata          147813 non-null  object
 9   StatusZakaza                 148621 non-null  object
 10  SposobDostavki               148621 non-null  object
 11  NomenklaturaKodRealAndZakaz  148621 non-null  object
 12  NomenklaturaRealizAndZakaz   148621 non-null  object
 13  VidNomenklatur

# Добавим удаленность от регионов

In [13]:
# Растояние от новосибирска до населенного пункта
otNovosibirska = {
    'Адыгея Респ': 4000,
    'Акмолинская область': 1200,
    'Актюбинская область': 2400,
    'Алматинская область': 1800,
    'Алтай Респ': 450,
    'Алтайский край': 230,
    'Амурская обл': 4300,
    'Архангельская обл': 3300,
    'Астраханская обл': 3600,
    'Астрахань': 3600,
    'Атырауская область': 2800,
    'Барнаул': 230,
    'Башкортостан Респ': 2000,
    'Белгород': 3600,
    'Белгородская обл': 3600,
    'Бишкек': 2000,
    'Благовещенск': 4300,
    'Брянская обл': 3700,
    'Бурятия Респ': 2300,
    'Владимирская обл': 3200,
    'Волгоградская обл': 3200,
    'Вологодская обл': 3000,
    'Воронежская обл': 3300,
    'Восточно-Казахстанская область': 740,
    'Дальневосточный ФО': None,
    'Еврейская Аобл': 4700,
    'Екатеринбург': 6400,
    'Иркутская обл': 1800,
    'Казань': 2500,
    'Казахстан': None,
    'Калининградская обл': None,
    'Калужская обл': 3600,
    'Камчатская обл': None,
    'Карагандинская область': 1000,
    'Кемерово': 260,
    'Кемеровская обл': 260,
    'Кировская обл': 2500,
    'Коми Респ': 3200,
    'Краснодар': 4000,
    'Краснодарский край': 4000,
    'Красноярск': 790,
    'Красноярский край': 790,
    'Курганская обл': 1200,
    'Кызылординская область': 2000,
    'Ленинградская обл': 4000,
    'Липецкая обл': 3300,
    'МОНГОЛИЯ': 2700,
    'Магаданская обл': 6700,
    'Мангистауская область': 3400,
    'Марий Эл Респ': 2700,
    'Мордовия Респ': 2900,
    'Москва': 3400,
    'Московская обл': 3400,
    'Мурманск': 4600,
    'Мурманская обл': 4600,
    'Ненецкий АО': 3900,
    'Нижегородская обл': 3000,
    'Новокузнецк': 370,
    'Новосибирск': 1,
    'Новосибирская обл': 1,
    'Омская обл': 650,
    'Оренбург': 2300,
    'Оренбургская обл': 2300,
    'Орловская обл': 3500,
    'Павлодарская обл': 670,
    'Пензенская обл': 2900,
    'Пермская обл': 2000,
    'Пермь': 2000,
    'Петрозаводск': 3900,
    'Приволжский ФО': None,
    'Приморский край': 3500,
    'Республика Крым': 4400,
    'Ростов-на-Дону': 3700,
    'Ростовская обл': 3700,
    'Рязанская обл': 3300,
    'Рязань': 3300,
    'Самарская обл': 2400,
    'Санкт-Петербург': 3900,
    'Саратовская обл': 2800,
    'Саха /Якутия/ Респ': 5000,
    'Сахалинская обл': 6100,
    'Свердловская обл': 1700,
    'Северо-Западный ФО': None,
    'Северо-Казахстанская область': 930,
    'Сибирский ФО': None,
    'Смоленская обл': 3700,
    'Ставрополь': 3800,
    'Ставропольский край': 3800,
    'Таймырский АО': 2700,
    'Тамбовская обл': 3100,
    'Татарстан Респ': 2400,
    'Теркестанская обл': None,
    'Томская обл': 900,
    'Тульская обл': 3500,
    'Тыва Респ': 1300,
    'Тюменская обл': 1300,
    'Удмуртская Респ': 2300,
    'Ульяновская обл': 2700,
    'Уральский ФО': None,
    'Хабаровский край': 5000,
    'Хакасия Респ': 1000,
    'Ханты-Мансийский - Югра АО': 1800,
    'Химки': 3400,
    'Центральный ФО': None,
    'Челябинск': 1600,
    'Челябинская обл': 1600,
    'Читинская обл': 2800,
    'Чувашская - Чувашия Респ': 2700,
    'Чукотский АО': None,
    'Южно-Сахалинск': 5900,
    'Якутск': 5000,
    'Ямало-Ненецкий АО': 2600,
    'Ярославская обл': 3200
}

In [14]:
# добавим данные о растоянии в DF
BaseDataFrame['RastoynieOtNSK'] = None
for number, series in BaseDataFrame.iterrows():
    BaseDataFrame.at[number, "RastoynieOtNSK"] = otNovosibirska.get(
        series['BiznesRegion'])

In [15]:
# Растояние от Москвы до населенного пункта
otMoscow = {
    'Адыгея Респ': 1400,
    'Акмолинская область': 2600,
    'Актюбинская область': 2000,
    'Алматинская область': 4000,
    'Алтай Респ': 3800,
    'Алтайский край': 3600,
    'Амурская обл': 7600,
    'Архангельская обл': 1200,
    'Астраханская обл': 1400,
    'Астрахань': 1400,
    'Атырауская область': 1700,
    'Барнаул': 3600,
    'Башкортостан Респ': 1500,
    'Белгород': 660,
    'Белгородская обл': 660,
    'Бишкек': 3700,
    'Благовещенск': 7800,
    'Брянская обл': 470,
    'Бурятия Респ': 5600,
    'Владимирская обл': 230,
    'Волгоградская обл': 950,
    'Вологодская обл': 810,
    'Воронежская обл': 650,
    'Восточно-Казахстанская область': 3700,
    'Дальневосточный ФО': None,
    'Еврейская Аобл': 8200,
    'Екатеринбург': 1800,
    'Иркутская обл': 5300,
    'Казань': 830,
    'Казахстан': None,
    'Калининградская обл': None,
    'Калужская обл': 230,
    'Камчатская обл': None,
    'Карагандинская область': 3000,
    'Кемерово': 3600,
    'Кемеровская обл': 3600,
    'Кировская обл': 970,
    'Коми Респ': 1700,
    'Краснодар': 1400,
    'Краснодарский край': 1400,
    'Красноярск': 4200,
    'Красноярский край': 4200,
    'Курганская обл': 2000,
    'Кызылординская область': 2800,
    'Ленинградская обл': 750,
    'Липецкая обл': 470,
    'МОНГОЛИЯ': 6000,
    'Магаданская обл': 10000,
    'Мангистауская область': 2600,
    'Марий Эл Респ': 780,
    'Мордовия Респ': 600,
    'Москва': 1,
    'Московская обл': 1,
    'Мурманск': 1900,
    'Мурманская обл': 1900,
    'Ненецкий АО': 2400,
    'Нижегородская обл': 470,
    'Новокузнецк': 3700,
    'Новосибирск': 3400,
    'Новосибирская обл': 3400,
    'Омская обл': 2800,
    'Оренбург': 1500,
    'Оренбургская обл': 1500,
    'Орловская обл': 410,
    'Павлодарская обл': 3100,
    'Пензенская обл': 610,
    'Пермская обл': 1400,
    'Пермь': 1400,
    'Петрозаводск': 1000,
    'Приволжский ФО': None,
    'Приморский край': 9100,
    'Республика Крым': 1800,
    'Ростов-на-Дону': 1100,
    'Ростовская обл': 1100,
    'Рязанская обл': 310,
    'Рязань': 310,
    'Самарская обл': 1000,
    'Санкт-Петербург': 710,
    'Саратовская обл': 940,
    'Саха /Якутия/ Респ': 8300,
    'Сахалинская обл': 9600,
    'Свердловская обл': 2000,
    'Северо-Западный ФО': None,
    'Северо-Казахстанская область': 2400,
    'Сибирский ФО': None,
    'Смоленская обл': 350,
    'Ставрополь': 1400,
    'Ставропольский край': 1400,
    'Таймырский АО': 6100,
    'Тамбовская обл': 500,
    'Татарстан Респ': 990,
    'Теркестанская обл': None,
    'Томская обл': 4200,
    'Тульская обл': 230,
    'Тыва Респ': 4700,
    'Тюменская обл': 2300,
    'Удмуртская Респ': 1300,
    'Ульяновская обл': 880,
    'Уральский ФО': None,
    'Хабаровский край': 8400,
    'Хакасия Респ': 4400,
    'Ханты-Мансийский - Югра АО': 2400,
    'Химки': 1,
    'Центральный ФО': None,
    'Челябинск': 1800,
    'Челябинская обл': 1800,
    'Читинская обл': 6300,
    'Чувашская - Чувашия Респ': 700,
    'Чукотский АО': None,
    'Южно-Сахалинск': 9300,
    'Якутск': 8300,
    'Ямало-Ненецкий АО': 3500,
    'Ярославская обл': 300
}

In [16]:
# добавим данные о растоянии в DF
BaseDataFrame['otMoscow'] = None
for number, series in BaseDataFrame.iterrows():
    BaseDataFrame.at[number, "otMoscow"] = otMoscow.get(series['BiznesRegion'])

In [17]:
BaseDataFrame['BiznesRegion'].fillna(value='Регион не указан', inplace=True)

In [18]:
print(BaseDataFrame['BiznesRegion'].value_counts())
print()
print(BaseDataFrame['BiznesRegion'].unique())

Новосибирская обл    43229
Кемеровская обл      17773
Алтайский край        7689
Приморский край       6673
Сибирский ФО          6278
                     ...  
Теркестанская обл        1
Пермь                    1
Рязань                   1
Краснодар                1
Новгородская обл         1
Name: BiznesRegion, Length: 114, dtype: int64

['Новосибирская обл' 'Тюменская обл' 'Павлодарская обл'
 'Саха /Якутия/ Респ' 'Приморский край' 'Хабаровский край' 'Читинская обл'
 'Алтайский край' 'Регион не указан' 'Бурятия Респ' 'Кемеровская обл'
 'Красноярский край' 'Сибирский ФО' 'Камчатская обл'
 'Северо-Казахстанская область' 'Томская обл' 'Ленинградская обл'
 'Омская обл' 'Иркутская обл' 'Ханты-Мансийский - Югра АО' 'Тыва Респ'
 'Свердловская обл' 'Амурская обл' 'Московская обл'
 'Восточно-Казахстанская область' 'Сахалинская обл' 'Еврейская Аобл'
 'Магаданская обл' 'Хакасия Респ' 'Карагандинская область'
 'Акмолинская область' 'Калужская обл' 'Татарстан Респ'
 'Башкортостан Респ' 'Челябин

# Трансформация данных

## Уберем пробелы из PartnerKod

In [19]:
for number, series in BaseDataFrame.iterrows():
    BaseDataFrame.at[number, 'PartnerKod'] = series['PartnerKod'].split(' ')[0]

## Обработка AsnovnManagerKod

In [20]:
# Заменили 'nan' на самого частого
BaseDataFrame['OsnovnoyManager'].fillna(
    value=BaseDataFrame['OsnovnoyManager'].describe().top, 
    inplace=True)

## Обработка отсутствущих значений KreditOrCash

In [21]:
print(BaseDataFrame['KreditIliPredoplata'].value_counts())
print()
print(BaseDataFrame['KreditIliPredoplata'].unique())

Нет    125233
Да      22580
Name: KreditIliPredoplata, dtype: int64

['Нет' None 'Да']


In [22]:
# Заменили "nan" на самого частого: 'Нет'
# nan только у заказов 2017 года (это заказпы из старой базы).
BaseDataFrame['KreditIliPredoplata'].fillna(value='Нет', inplace=True)

In [23]:
print(BaseDataFrame['KreditIliPredoplata'].value_counts())
print()
print(BaseDataFrame['KreditIliPredoplata'].unique())

Нет    126041
Да      22580
Name: KreditIliPredoplata, dtype: int64

['Нет' 'Да']


In [24]:
# Удалим, т.к. нет необхожимости в такой глубине
del BaseDataFrame['GrafikOplaty']

## Обработка отсутствующих значений Status
### Уберем строки у которых статусы = 'На согласовании'.

In [25]:
print(BaseDataFrame['StatusZakaza'].value_counts())
print()
print(BaseDataFrame['StatusZakaza'].unique())

Закрыт                      145176
К выполнению / В резерве      3041
На согласовании                404
Name: StatusZakaza, dtype: int64

['Закрыт' 'К выполнению / В резерве' 'На согласовании']


In [26]:
# У заказов "На согласовании" не может быть реализаций в структуре, поэтому удалим из дальнейшего анализа
BaseDataFrame.drop(index=BaseDataFrame.loc[BaseDataFrame['StatusZakaza'] ==
                                           'На согласовании'].index,
                   axis=0,
                   inplace=True)

In [27]:
print(BaseDataFrame['StatusZakaza'].value_counts())
print()
print(BaseDataFrame['StatusZakaza'].unique())

Закрыт                      145176
К выполнению / В резерве      3041
Name: StatusZakaza, dtype: int64

['Закрыт' 'К выполнению / В резерве']


### Т.к. у заказа в статусе 'К выполнению / В резерве' может быть оплаченная реализация (т.е частичная отгрузка), то выделим заказы с присутствующими реализациями. Их нужно оставить в обучающем наборе.

In [28]:
# Добавим новый столбец, как признак состоявшейся продажи
BaseDataFrame['ПродажаСостоялась'] = 'Нет'

In [29]:
# Перекодируем значение
for number, series in BaseDataFrame.iterrows():
    if type(series['RealizSsilka']) == str:
        BaseDataFrame.at[number, 'ПродажаСостоялась'] = 'Да'

In [30]:
BaseDataFrame['ПродажаСостоялась'].value_counts()

Да     81869
Нет    66348
Name: ПродажаСостоялась, dtype: int64

In [31]:
# Получим индексы заказав
IndexNETRealiz = BaseDataFrame.loc[BaseDataFrame['ПродажаСостоялась'] == 'Нет'].index
IndexOtkritZakaz = BaseDataFrame.loc[BaseDataFrame['StatusZakaza']
                                     == 'К выполнению / В резерве'].index

In [32]:
IndexUdalit = IndexNETRealiz.intersection(IndexOtkritZakaz)
IndexUdalit

Int64Index([  1186,  75143,  75144,  75145,  75146,  75147,  75148,  75149,
             75150,  75151,
            ...
            148611, 148612, 148613, 148614, 148615, 148616, 148617, 148618,
            148619, 148620],
           dtype='int64', length=2201)

### Удалим открытые заказы по которым нет реализаций

In [33]:
BaseDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148217 entries, 0 to 148620
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   AktVipRabot                  2390 non-null    object        
 1   Data                         2390 non-null    datetime64[ns]
 2   SummaAktaVipRabot            2390 non-null    float64       
 3   Quantity                     2390 non-null    float64       
 4   KodStrokiAkt                 2390 non-null    float64       
 5   ZakazKlienta                 2390 non-null    object        
 6   NomenklaturaKod              2390 non-null    object        
 7   Nomenklatura                 2390 non-null    object        
 8   ZakazData                    148217 non-null  datetime64[ns]
 9   PartnerKod                   148217 non-null  object        
 10  OsnovnoyManager              148217 non-null  object        
 11  BiznesRegion              

In [34]:
# Удалим данные по открытым заказам
BaseDataFrame.drop(index = IndexUdalit, axis = 0, inplace=True)

In [35]:
# сбросим имеющиеся индексы и воссоздадим по порядку заново
BaseDataFrame.reset_index(inplace=True)
del BaseDataFrame['index']

In [36]:
BaseDataFrame['ПродажаСостоялась'].value_counts()

Да     81869
Нет    64147
Name: ПродажаСостоялась, dtype: int64

In [37]:
BaseDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146016 entries, 0 to 146015
Data columns (total 43 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   AktVipRabot                  2375 non-null    object        
 1   Data                         2375 non-null    datetime64[ns]
 2   SummaAktaVipRabot            2375 non-null    float64       
 3   Quantity                     2375 non-null    float64       
 4   KodStrokiAkt                 2375 non-null    float64       
 5   ZakazKlienta                 2375 non-null    object        
 6   NomenklaturaKod              2375 non-null    object        
 7   Nomenklatura                 2375 non-null    object        
 8   ZakazData                    146016 non-null  datetime64[ns]
 9   PartnerKod                   146016 non-null  object        
 10  OsnovnoyManager              146016 non-null  object        
 11  BiznesRegion              

## Обработаем пропущенные значения столбца KodeOfTNVED 

In [38]:
# Пропущенные значения заменили на "НеУказано"
BaseDataFrame['KodTNVED'].fillna(value='НеУказано', inplace=True)

In [39]:
# Заменим значение '<>' на 'НеУказано'
f = {'<>':'НеУказано'}
BaseDataFrame = BaseDataFrame.replace({'KodTNVED': f})

In [40]:
BaseDataFrame['KodTNVED'].unique()

array(['8424100000', '8424900000', '7609000000', '9403105809',
       '5909001000', 'НеУказано', '4821901000', '3813000000',
       '8201500000', '8201900009', '7323940000', '8201400000',
       '7310100000', '7019590000', '9403109300', '7307191000',
       '7307998009', '8481807900', '7326909807', '8424890009',
       '4008219000', '6401921000', '7326200009', '3006500000',
       '4015190000', '3214101001', '8310000000', '3923100000',
       '8481806100', '8504405500', '8513100000', '6815109008',
       '4202929800', '5607509000', '7326903000', '3102109000',
       '9402900000', '6307200000', '3824997000', '3926909709',
       '6506101000', '3208209009', '6210400000', '6506108000',
       '8481808508', '8507302009'], dtype=object)

## Обработка столбца TypeOfPrice

### Заменим пропуски в столбце TypeOfPrice на 'Произвольная'

In [41]:
print(BaseDataFrame['VidCeny'].value_counts())
print()
print(BaseDataFrame['VidCeny'].unique())

От 50 тыс. руб.                              115090
От 5 тыс. руб.                                11707
От 50 тыс. руб. КЗ                             4968
Розница                                        4919
От 50 тыс. руб. КЗ (по 31 декабря 2018г.)      4027
От 5 тыс. руб. КЗ (по 31 декабря 2018г.)        171
От 5 тыс. руб. КЗ                               132
Цена из Ярославля                                24
Name: VidCeny, dtype: int64

[None 'От 50 тыс. руб. КЗ (по 31 декабря 2018г.)' 'От 50 тыс. руб.'
 'От 5 тыс. руб.' 'Розница' 'От 5 тыс. руб. КЗ (по 31 декабря 2018г.)'
 'От 50 тыс. руб. КЗ' 'От 5 тыс. руб. КЗ' 'Цена из Ярославля']


In [42]:
BaseDataFrame['VidCeny'].fillna(value='Произвольная', inplace=True)

In [43]:
BaseDataFrame['VidCeny'].value_counts()

От 50 тыс. руб.                              115090
От 5 тыс. руб.                                11707
Произвольная                                   4978
От 50 тыс. руб. КЗ                             4968
Розница                                        4919
От 50 тыс. руб. КЗ (по 31 декабря 2018г.)      4027
От 5 тыс. руб. КЗ (по 31 декабря 2018г.)        171
От 5 тыс. руб. КЗ                               132
Цена из Ярославля                                24
Name: VidCeny, dtype: int64

### Объеденим старые типы цен КЗ с новыми

In [44]:
f = {
    'От 50 тыс. руб. КЗ (по 31 декабря 2018г.)': 'От 50 тыс. руб. КЗ',
    'От 5 тыс. руб. КЗ (по 31 декабря 2018г.)': 'От 5 тыс. руб. КЗ'
}
BaseDataFrame.replace({'VidCeny': f}, inplace=True)

In [45]:
BaseDataFrame['VidCeny'].value_counts()

От 50 тыс. руб.       115090
От 5 тыс. руб.         11707
От 50 тыс. руб. КЗ      8995
Произвольная            4978
Розница                 4919
От 5 тыс. руб. КЗ        303
Цена из Ярославля         24
Name: VidCeny, dtype: int64

## Обработка столбца StavkaNDS

In [46]:
print(BaseDataFrame['StavkaNDS'].value_counts())
print()
print(BaseDataFrame['StavkaNDS'].unique())

20%        76146
18%        60064
0%          8813
Без НДС      992
20/120         1
Name: StavkaNDS, dtype: int64

['20%' '18%' '0%' 'Без НДС' '20/120']


### Объеденим: '20%' : '18%' и 'Без НДС':'0%' и '20/120':'20%'

In [47]:
f = {'18%':'20%', 'Без НДС':'0%', '20/120':'20%'}
BaseDataFrame = BaseDataFrame.replace({'StavkaNDS': f})

In [48]:
BaseDataFrame['StavkaNDS'].value_counts()

20%    136211
0%       9805
Name: StavkaNDS, dtype: int64

## Обработаем столбцы ProcentRuchnoySkidki и ProcentAvtomatichSkidki

In [49]:
BaseDataFrame['ItogoProcentSkidki'] = BaseDataFrame['ProcentRuchSkidki'] + \
    BaseDataFrame['ProcentAvtomSkidki']

In [50]:
# Удалим старые столбцы
del BaseDataFrame['ProcentRuchSkidki'] 
del BaseDataFrame['ProcentAvtomSkidki']

## Обработаем столбец ReasonOfCancelled

In [51]:
print(BaseDataFrame['PrichinaOtmeny'].value_counts())
print()
print(BaseDataFrame['PrichinaOtmeny'].unique())

Не известны                                                            23486
Причина НЕ выяснялась                                                  19094
Проверка цен                                                            8119
Перекуп не сработал                                                     5643
Заказ обновлен на другой счет (дату)                                    1155
Дублирующаяся заявка                                                    1118
Зависшие суммы, количество или услуги (только для Администратора)       1023
Не согласован                                                            586
Не устроил срок поставки                                                 472
Не прошли по цене из-за географии покупателя                             262
Моя вина                                                                 157
Выставляли аналог                                                        136
Не успели привезти                                                       129

### Заменим 'nan' на 'Продажа состоялась'.
#### Это возможно сделать, т.к. мы удалили все заказы "Открытые" заказы по которым нет реализаций и статус номенклатуры не равен "К обеспечению" или "Обеспечивать обособлено"

In [52]:
BaseDataFrame['PrichinaOtmeny'].fillna(value='ПродажаСостоялась', inplace=True)

In [53]:
BaseDataFrame['ПродажаСостоялась'].value_counts()

Да     81869
Нет    64147
Name: ПродажаСостоялась, dtype: int64

In [54]:
print(BaseDataFrame['PrichinaOtmeny'].value_counts())

ПродажаСостоялась                                                      84271
Не известны                                                            23486
Причина НЕ выяснялась                                                  19094
Проверка цен                                                            8119
Перекуп не сработал                                                     5643
Заказ обновлен на другой счет (дату)                                    1155
Дублирующаяся заявка                                                    1118
Зависшие суммы, количество или услуги (только для Администратора)       1023
Не согласован                                                            586
Не устроил срок поставки                                                 472
Не прошли по цене из-за географии покупателя                             262
Моя вина                                                                 157
Выставляли аналог                                                        136

## Обработаем BiznesRegion

In [55]:
print(BaseDataFrame['BiznesRegion'].value_counts())
print()
print(BaseDataFrame['BiznesRegion'].unique())

Новосибирская обл      42443
Кемеровская обл        17540
Алтайский край          7633
Приморский край         6595
Сибирский ФО            6135
                       ...  
Краснодар                  1
Калининградская обл        1
Теркестанская обл          1
Рязань                     1
Новгородская обл           1
Name: BiznesRegion, Length: 114, dtype: int64

['Новосибирская обл' 'Тюменская обл' 'Павлодарская обл'
 'Саха /Якутия/ Респ' 'Приморский край' 'Хабаровский край' 'Читинская обл'
 'Алтайский край' 'Регион не указан' 'Бурятия Респ' 'Кемеровская обл'
 'Красноярский край' 'Сибирский ФО' 'Камчатская обл'
 'Северо-Казахстанская область' 'Томская обл' 'Ленинградская обл'
 'Омская обл' 'Иркутская обл' 'Ханты-Мансийский - Югра АО' 'Тыва Респ'
 'Свердловская обл' 'Амурская обл' 'Московская обл'
 'Восточно-Казахстанская область' 'Сахалинская обл' 'Еврейская Аобл'
 'Магаданская обл' 'Хакасия Респ' 'Карагандинская область'
 'Акмолинская область' 'Калужская обл' 'Татарстан Респ'
 'Башко

In [56]:
BaseDataFrame['BiznesRegion'].fillna(value='Регион не указан', inplace=True)

## Заменим пропуски в переменных Растояния

In [57]:
BaseDataFrame['RastoynieOtNSK'].fillna(value='999999', inplace=True)
BaseDataFrame['otMoscow'].fillna(value='999999', inplace=True)

# Сохраним данные

In [58]:
with pd.ExcelWriter(
        f'3-Финальные данные от {datetime.today().strftime("%Y-%m-%d")}.xlsx'
) as writer:
    BaseDataFrame.reset_index(drop=True).to_excel(
        writer, sheet_name='Данные для анализа')